# State Population

This notebook demonstrates how to model state and population effects when fitting models. See `xforms Demo` for details on xforms.

In [1]:
from pathlib import Path

import nems.uri
import nems.xforms as xforms
import nems.recording as recording

[nems.configs.defaults INFO] Saving log messages to /tmp/nems\NEMS 2020-05-28 135411.log


## Configuration

In [2]:
# get the data and results paths
results_dir = nems.get_setting('NEMS_RESULTS_DIR')
signals_dir = nems.get_setting('NEMS_RECORDINGS_DIR')

In [3]:
# download some demo data
recording.get_demo_recordings(signals_dir, 'TAR010c.NAT.fs100.tgz')
datafile = Path(signals_dir) / 'TAR010c.NAT.fs100.tgz'

## Data Loading and Preprocessing

In [4]:
exptid = 'TAR010c'
batch = 271
cellid = 'TAR010c-18-2'

In [5]:
modelspecname = 'stategain.SxN'

## Generate the Modelspec

In [6]:
meta = {'cellid': cellid, 'batch': batch, 'modelname': modelspecname,
        'recording': exptid}

state_signals = ["pupil", "active", "population", "pupil_x_population", "active_x_population"]
jk_kwargs = {'njacks': 5}
xforms_init_context = {'cellid': cellid, 'batch': int(batch)}
xforms_init_context['keywordstring'] = modelspecname
xforms_init_context['meta'] = meta
xforms_init_context['recording_uri_list'] = [str(datafile)]

In [7]:
xfspec = []

In [8]:
xfspec.append(['nems.xforms.init_context', xforms_init_context])
xfspec.append(['nems.xforms.load_recordings', {"save_other_cells_to_state": "population"}])
xfspec.append(['nems.xforms.make_state_signal',
              {'state_signals': state_signals, 'permute_signals': []}])
xfspec.append(["nems.xforms.mask_all_but_correct_references", {}])
xfspec.append(["nems.xforms.generate_psth_from_resp", {"smooth_resp": False, "use_as_input": True, "epoch_regex": "^STIM_"}])
xfspec.append(['nems.xforms.init_from_keywords', {}])
xfspec.append(['nems.xforms.mask_for_jackknife', jk_kwargs])
xfspec.append(['nems.xforms.fit_basic', {}])

In [9]:
# test and visualize
xfspec.append(['nems.xforms.predict', {}])
xfspec.append(['nems.xforms.add_summary_statistics', {}])
xfspec.append(['nems.xforms.plot_summary', {}])

## Run the Analysis

In [ ]:
ctx = {}
for xfa in xfspec:
    ctx = xforms.evaluate_step(xfa, ctx)